In [9]:
import random
import math

## Enemies

In [33]:
from dataclasses import dataclass

@dataclass
class Creature:
    hp: int
    armor: tuple[int, int]
    accuracy: int
    evasion: int
    damage: tuple[int, int]

class Creatures:
    rat = Creature(8,(0,1), 8, 2, (1,4))
    snake = Creature(4,(0,0), 10, 25, (1,4))
    gnoll = Creature(12,(0,2), 10, 4, (1,6))
    crab = Creature(15,(0,4), 12, 5, (1,7))
    slime = Creature(20,(0,0), 12, 5, (2,5))
    flies = Creature(50,(0,0), 10, 5, (1,4))
    skeleton = Creature(25,(0,5), 12, 9, (2,10))
    thief = Creature(20,(0,3), 12, 12, (1,10))
    necromancer = Creature(40,(0,5), 12, 14, (2,10))
    guard = Creature(40,(0,7), 12, 10, (4,12))
    dm100 = Creature(20,(0,4), 11, 8, (2,8))
    bat = Creature(30,(0,4), 16, 15, (5,18))
    brute = Creature(40,(0,8), 20, 15, (5,25))
    shaman = Creature(35,(0,6), 18, 15, (5,10))
    spider = Creature(50,(0,6), 20, 14, (10,20))
    dm200 = Creature(80,(0,8), 25, 8, (10,25))
    ghoul = Creature(45,(0,4), 24, 20, (16,22))
    elemental = Creature(65,(0,5), 25, 20, (20,25))
    warlock = Creature(70,(0,8), 25, 18, (16,22))
    monk = Creature(68,(0,2), 30, 30, (12,25))
    golem = Creature(120,(0,12), 28, 15, (25,30))
    succubus = Creature(80,(0,10), 40, 25, (25,30))
    eye = Creature(100,(0,10), 30, 20, (20,30))
    scorpio = Creature(110,(0,16), 36, 24, (30,40))
    demon = Creature(60,(0,4), 30, 22, (15,25))
    goo = Creature(120,(0,2), 10, 8, (1,8))
    goo_enraged = Creature(120,(0,2), 15, 8 * 1.5, (1,12))
    tengu = Creature(250,(0,5), 20, 15, (6,12))
    dm300 = Creature(400,(0,10), 20, 15, (15,25))
    dk = Creature(450,(0,10), 26, 22, (15,25))
    fist = Creature(300,(0,15), 36, 20, (18,36))


## Equipment

In [11]:
from dataclasses import dataclass

@dataclass
class Armor:
    min: int
    max: int

class Armors:
    cloth = Armor(0, 2)
    leather = Armor(0, 3)
    mail = Armor(0, 4)
    scale = Armor(0, 5)
    plate = Armor(0, 6)

## Augmentation factor

In [12]:
class Augmentation:
    UNAUGMENTED = (0, 0)
    DEFENSE = (-2, 1)
    EVASION = (2, -1)

## Helper functions

In [ ]:
def calc_hit_rate(accuracy, evasion):
    if evasion >= accuracy:
        return accuracy / (2 * evasion)
    else:
        return 1 - evasion / (2 * accuracy)



def calc_percent_change(original, new):
    return (new - original)/original * 100

def calc_final_damage(damage, defense):
    return max(0, damage - defense)

def calc_defense_interval_fima(base_armor, armor_level, armor_augment):
    base_armor_max = base_armor[1]
    max_defense = base_armor_max + armor_level + armor_augment[1] * (armor_level + 2)
    return (base_armor[0], max_defense)

def calc_final_evasion(base_evasion, armor_level, armor_augment, evasion_multiplier):
    return base_evasion * evasion_multiplier + armor_augment[0] * (armor_level + 2)

def triangular_discrete(low, high):
    # min + (int)((Float() + Float()) * (max - min + 1) / 2f);
    u1 = random.uniform(0,1)
    u2 = random.uniform(0,1)
    return low + math.floor((u1 + u2) * (high - low + 1) / 2) 

def simulate_damage(defense_interval, damage_interval, hit_chance, num_iterations):
    total_damage = 0
    for _ in range(num_iterations):
        defense_roll = triangular_discrete(defense_interval[0], defense_interval[1])
        damage_roll = triangular_discrete(damage_interval[0], damage_interval[1])
        evasion_roll = 1 if random.uniform(0, 1) < hit_chance else 0
        damage = evasion_roll * calc_final_damage(damage_roll, defense_roll)
        total_damage += damage
    avg_damage = total_damage / num_iterations
    return avg_damage

def simulate(accuracy, damage_interval, base_armor, armor_level, hero_level, evasion_multiplier, augmentation, num_simulations):
    base_evasion = hero_level + 4
    final_evasion = calc_final_evasion(base_evasion, armor_level, augmentation, evasion_multiplier)
    final_defense_interval = calc_defense_interval_fima(base_armor, armor_level, augmentation)
    hit_chance = calc_hit_rate(accuracy, final_evasion)
    avg_damage = simulate_damage(final_defense_interval, damage_interval, hit_chance, num_simulations)
    return avg_damage


## Augmentation simulation

### Variables

In [14]:
# configure variables here
creature = Creatures.bat
armor = Armors.leather
armor_level = 3
hero_level = 13
evasion_multiplier = 1
num_simulations = 1000000

### Compare augmentations

In [15]:
accuracy = creature.accuracy
damage_interval = creature.damage
base_armor = (armor.min, armor.max)

avg_damage_unaug = simulate(accuracy, damage_interval, base_armor, armor_level, hero_level, evasion_multiplier, Augmentation.UNAUGMENTED, num_simulations)
avg_damage_defense = simulate(accuracy, damage_interval, base_armor, armor_level, hero_level, evasion_multiplier, Augmentation.DEFENSE, num_simulations)
avg_damage_evasion = simulate(accuracy, damage_interval, base_armor, armor_level, hero_level, evasion_multiplier, Augmentation.EVASION, num_simulations)

percent_change_defense = calc_percent_change(avg_damage_unaug, avg_damage_defense)
percent_change_evasion = calc_percent_change(avg_damage_unaug, avg_damage_evasion)

print(f"Defense augment percent change: {percent_change_defense}")
print(f"Evasion augment percent change: {percent_change_evasion}")

Defense augment percent change: 18.85816722911853
Evasion augment percent change: -18.520646035446862


### Compare Clover Number of hits

In [76]:
import numpy as np

def calc_clover_proc_chance(level):
    return 0.25 * (level + 1)

def get_clover_damage(damage_range, extra_strength, clover_level):
    (min_damage, max_damage) = damage_range
    normal_damage = triangular_discrete(min_damage, max_damage) + triangular_discrete(0, extra_strength)
    proc_chance = calc_clover_proc_chance(clover_level)
    return random.choices([normal_damage, min_damage, max_damage + extra_strength], weights=[1-proc_chance, 2/5 * proc_chance, 3/5 * proc_chance])[0]

def simulate_fight(hp, defense_range, damage_range, extra_strength, clover_level):
    curr_hp = hp
    num_hits = 0
    while(curr_hp > 0):
        damage_roll = get_clover_damage(damage_range, extra_strength, clover_level)
        (defense_low, defense_high) = defense_range
        defense_roll = triangular_discrete(defense_low, defense_high)
        curr_hp -= calc_final_damage(damage_roll, defense_roll)
        num_hits += 1
    return num_hits


def simulate_clover_efficacy(enemy, damage_range, extra_strength, num_simulations):
    data_points_clover0 = []
    data_points_clover1 = []
    data_points_clover2 = []
    data_points_clover3 = []
    data_points_normal = []
    for _ in range(num_simulations):
        num_hits = simulate_fight(enemy.hp, enemy.armor, damage_range, extra_strength, 0)
        data_points_clover0.append(num_hits)
    for _ in range(num_simulations):
        num_hits = simulate_fight(enemy.hp, enemy.armor, damage_range, extra_strength, 1)
        data_points_clover1.append(num_hits)
    for _ in range(num_simulations):
        num_hits = simulate_fight(enemy.hp, enemy.armor, damage_range, extra_strength, 2)
        data_points_clover2.append(num_hits)
    for _ in range(num_simulations):
        num_hits = simulate_fight(enemy.hp, enemy.armor, damage_range, extra_strength, 3)
        data_points_clover3.append(num_hits)
    for _ in range(num_simulations):
        num_hits = simulate_fight(enemy.hp, enemy.armor, damage_range, extra_strength, -1)
        data_points_normal.append(num_hits)

    avg_normal = np.average(data_points_normal)
    avg0 = np.average(data_points_clover0)
    avg1 = np.average(data_points_clover1)
    avg2 = np.average(data_points_clover2)
    avg3 = np.average(data_points_clover3)

    var_normal = np.var(data_points_normal)
    var0 = np.var(data_points_clover0)
    var1 = np.var(data_points_clover1)
    var2 = np.var(data_points_clover2)
    var3 = np.var(data_points_clover3)
    
    
    print(f'Num simulations: {num_simulations}')
    print('Average number of hits')
    print(f'Normal {avg_normal}')
    print(f'Clover +0 {avg0}')
    print(f'Clover +1 {avg1}')
    print(f'Clover +2 {avg2}')
    print(f'Clover +3 {avg3}')
    print(f'Variance:')
    print(f'Normal {var_normal}')
    print(f'Clover +0 {var0}')
    print(f'Clover +1 {var1}')
    print(f'Clover +2 {var2}')
    print(f'Clover +3 {var3}')
    print(f'% change:')
    print(f'Clover +0 {calc_percent_change(avg_normal, avg0)}')
    print(f'Clover +1 {calc_percent_change(avg_normal, avg1)}')
    print(f'Clover +2 {calc_percent_change(avg_normal, avg2)}')
    print(f'Clover +3 {calc_percent_change(avg_normal, avg3)}')
    

In [ ]:
# Config variables
damage_range = (8,62)
extra_strength = 2
enemy = Creatures.elemental
num_simulations = 500000

In [97]:
simulate_clover_efficacy(enemy, damage_range, extra_strength, num_simulations)

Num simulations: 500000
Average number of hits
Normal 2.724186
Clover +0 2.73914
Clover +1 2.775728
Clover +2 2.851686
Clover +3 2.977816
Variance:
Normal 0.39974463740399996
Clover +0 0.6599640603999994
Clover +1 0.8657620700159999
Clover +2 1.0024089574040005
Clover +3 0.9551518701439998
% change:
Clover +0 0.5489346175334544
Clover +1 1.8920147155884355
Clover +2 4.680297160326055
Clover +3 9.31030406881174
